In [ ]:
Equations are given in the format A / B = k, where A and B are variables represented as strings, and k is a real number (floating point number). Given some queries, return the answers. If the answer does not exist, return -1.0.

Example:
Given a / b = 2.0, b / c = 3.0.
queries are: a / c = ?, b / a = ?, a / e = ?, a / a = ?, x / x = ? .
return [6.0, 0.5, -1.0, 1.0, -1.0 ].

The input is: vector<pair<string, string>> equations, vector<double>& values, vector<pair<string, string>> queries , where equations.size() == values.size(), and the values are positive. This represents the equations. Return vector<double>.

According to the example above:

equations = [ ["a", "b"], ["b", "c"] ],
values = [2.0, 3.0],
queries = [ ["a", "c"], ["b", "a"], ["a", "e"], ["a", "a"], ["x", "x"] ]. 
 

The input is always valid. You may assume that evaluating the queries will result in no division by zero and there is no contradiction.

来源：力扣（LeetCode）
链接：https://leetcode-cn.com/problems/evaluate-division
著作权归领扣网络所有。商业转载请联系官方授权，非商业转载请注明出处。

In [20]:
#Floyd
#https://www.zhihu.com/search?type=content&q=Floyd%E7%AE%97%E6%B3%95
from typing import List
class Solution:
    def calcEquation(self, equations: List[List[str]], values: List[float], queries: List[List[str]]) -> List[float]:
        store={}
        index=0
        en=len(equations)
        inf=float('inf')
        for i in range(en):
            a,b=equations[i]
            if a not in store:
                store[a]=index
                index+=1    
            if b not in store: 
                store[b]=index
                index+=1  

        graph=[[inf]*index for _ in range(index)]

        #邻接矩阵
        for i in range(en):
            a,b=equations[i]
            ia,ib=store[a],store[b]
            v=values[i]
            graph[ia][ib]=v
            graph[ib][ia]=1/v

        for k in range(index):#中转站0~k
            for i in range(index): #i为起点
                for j in range(index): #j为终点
                    #松弛条件
                    if graph[i][j]==inf and graph[k][j]!=inf and graph[i][k]!=inf:
                        graph[i][j]=graph[i][k]*graph[k][j]
        
        results=[]
        for q in queries:
            if q[0] not in store or q[1] not in store:
                print(q)
                results.append(-1.0)
                continue
            ia=store[q[0]]
            ib=store[q[1]]
            r=graph[ia][ib]
            r=r if r!=inf else -1.0
            results.append(r)
        print(store,index,graph,results)
        return results

equations = [ ["a", "b"], ["b", "c"] ]
values = [2.0, 3.0]
queries = [ ["a", "c"], ["b", "a"], ["a", "e"], ["a", "a"], ["x", "x"] ]
Solution().calcEquation(equations,values,queries)


['a', 'e']
['x', 'x']
{'a': 0, 'b': 1, 'c': 2} 3 [[1.0, 2.0, 6.0], [0.5, 1.0, 3.0], [0.16666666666666666, 0.3333333333333333, 1.0]] [6.0, 0.5, -1.0, 1.0, -1.0]


[6.0, 0.5, -1.0, 1.0, -1.0]

In [65]:
#并查集
from typing import List
class Node:
    def __init__(self,v):
        self.v=v
        self.parent=self

def find(node:Node) -> Node:
    while node!=node.parent:
        node=node.parent
    return node

def union(node1,node2,val,mp):
    root1=find(node1)
    root2=find(node2)
    if root1!=root2:
        ratio=node2.v*val/node1.v
        for m in mp:
            if root1==find(mp[m]):
                mp[m].v*=ratio
        root1.parent=root2


class Solution:
    def calcEquation(self, equations: List[List[str]], values: List[float], queries: List[List[str]]) -> List[float]:
        mp={}
        n=len(equations)
        for i in range(n):
            e=equations[i]
            v=values[i]
            # if e[0] not in mp and e[1] not in mp:
            #     mp[e[0]]=v
            #     mp[e[1]]=1.0
            # elif e[0] not in mp and e[1] in mp:
            #     mp[e[0]]=v*mp[e[1]]
            # elif e[0] in mp and e[1] not in mp:
            #     mp[e[1]]=mp[e[0]]/v
            # else:
            #     #不用并查集的话，两个分离的图无法合并，无法更新参数
            #     pass
            if e[0] not in mp:
                mp[e[0]]=Node(v)
            if e[1] not in mp:
                mp[e[1]]=Node(1.0)
            
            union(mp[e[0]],mp[e[1]],v,mp)
        
        results=[]
        for q in queries:
            if q[0] not in mp or q[1] not in mp:
                results.append(-1.0)
                continue
            ia=mp[q[0]]
            ib=mp[q[1]]
            results.append(ia.v/ib.v)
        return results
            
equations = [ ["a", "b"], ["b", "c"] ]
values = [2.0, 3.0]
queries = [ ["a", "c"], ["b", "a"], ["a", "e"], ["a", "a"], ["x", "x"] ]
Solution().calcEquation(equations,values,queries)     

[6.0, 0.5, -1.0, 1.0, -1.0]

In [56]:
#DFS and BFS
from typing import List
class Solution:
    
    def dfs(self,edges,visited,begin,end):
        if begin not in edges or end not in edges:
            return self.inf
        if begin in visited:
            return self.inf
        
        if end in edges[begin]:
            return edges[begin][end]
        
        visited.add(begin)
        for edge in edges[begin]:
            r=self.dfs(edges,visited,edge,end)
            if r!=self.inf:
                return edges[begin][edge]*r
            else:
                continue
        return self.inf
    
    def bfs(self,edges,visited,begin,end):
        if begin not in edges or end not in edges:
            return self.inf

        from collections import deque
        queue=deque()
        queue.append((begin,1))
        while queue:
            cur=queue.popleft()
            if cur[0] in visited:
                continue
            if cur[0]==end:
                return cur[1]
            visited.add(cur[0])
            for edge in edges[cur[0]]:
                queue.append((edge,cur[1]*edges[cur[0]][edge]))

        return self.inf

    def calcEquation(self, equations: List[List[str]], values: List[float], queries: List[List[str]]) -> List[float]:
        #边集
        edges={}
        index=0
        en=len(equations)
        self.inf=float('inf')
        for i in range(en):
            a,b=equations[i]
            v=values[i]
            if a not in edges:
                edges[a]={b:v} 
            else:
                edges[a][b]=v  
            if b not in edges: 
                edges[b]={a:1/v}
            else:
                edges[b][a]=1/v 
        
        results=[]
        for q in queries:
            visited=set()
            # r = self.dfs(edges, visited, q[0], q[1])
            r = self.bfs(edges, visited, q[0], q[1])
            if r!=self.inf:
                edges[q[0]][q[1]] = r
                edges[q[1]][q[0]] = 1 / r
                results.append(r)
            else:
                results.append(-1.0)
        return results
           

equations = [ ["a", "b"], ["b", "c"] ]
values = [2.0, 3.0]
queries = [ ["a", "c"], ["b", "a"], ["a", "e"], ["a", "a"], ["x", "x"] ]
Solution().calcEquation(equations,values,queries)


[6.0, 0.5, -1.0, 1, -1.0]